In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-02-21 12:48:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.006s  

2025-02-21 12:48:14 (170 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt','r',encoding='utf-8') as f:
  text = f.read()

In [ ]:
print(f'Length of the dataset is {len(text)}')

Length of the dataset is 1115394


In [ ]:
#insepcting the dataset
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f"Length of the Vocab is {vocab_size}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Length of the Vocab is 65


In [ ]:
itos = { i:k for i,k in enumerate(chars)}
stoi = { k:i for i,k in enumerate(chars)}
encode = lambda s: [stoi[k] for k in list(s)]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("hey there!"))
print(decode(encode("hey there!")))

[46, 43, 63, 1, 58, 46, 43, 56, 43, 2]
hey there!


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
#let's create train and val split
n = int(0.9*len(data))
train_data = data[:n] #90% data -> train dataa
val_data = data[n:]#rest 10% data -> val data
print(f'Train data shape : {train_data.shape}')
print(f'Val Data shape : {val_data.shape}')

Train data shape : torch.Size([1003854])
Val Data shape : torch.Size([111540])


In [ ]:
## create batches of data
block_size = 8                                                                  ## hyperparameter 1: block_size -> context length -> time dimensions
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
y= train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"When input is {context}, the target is : {target}")

When input is tensor([18]), the target is : 47
When input is tensor([18, 47]), the target is : 56
When input is tensor([18, 47, 56]), the target is : 57
When input is tensor([18, 47, 56, 57]), the target is : 58
When input is tensor([18, 47, 56, 57, 58]), the target is : 1
When input is tensor([18, 47, 56, 57, 58,  1]), the target is : 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is : 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is : 58


In [ ]:
## here's the code for making batches

torch.manual_seed(1337)
batch_size = 4                                                                  ## Hyperparameter 2: Batch Size
block_size = 8

def get_batch(split):
  data = train_data if split=="train" else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x.to(device), y.to(device)

xb,yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

NameError: name 'device' is not defined

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size, embed_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(num_embeddings = vocab_size, embedding_dim=embed_size)

  def forward(self,idx, targets=None):
    #idx and targets are both (B,T) tensor of integers
    logits = self.token_embedding_table(idx)  #(B,T,C)
    if targets is None:
      loss = None
    else :
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits , loss

  def generate(self, idx, max_new_tokens):
     #idx is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      #get the predictions
      logits, loss = self(idx)  ## similar to self.forward(idx)  as __call__ function, 'calls' forward internally
      ## logits : shape -> (B,T,C)
      logits = logits[:, -1,:] #becomes (B,C)
      #apply softmax to get the probabilities
      probs = F.softmax(logits, dim=-1) ## (B,C)
      idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
      idx = torch.cat((idx, idx_next), dim=1) #(B,T+1)
    return idx



model = BigramLanguageModel(vocab_size = vocab_size, embed_size = vocab_size).to(device)
logits, loss = model(xb, yb)
print(logits.shape)
print("loss = ",loss.item())

print(decode(model.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
loss =  4.65185546875

MFKL?HYbb&qfn$Y:y?utqFFSQfAEL
.OJDemHR.
MzhrJL?KzG:NljawHwaoGk,SKbCK&qQS
UUq.xcD 3K-SeFRODSQM!B!xgj'


In [ ]:
## Let's train this mini model
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [ ]:
batch_size = 32

for epoch in range(10000):
  xb, yb = get_batch("train")
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  if epoch%1000==0:
    print(loss.item())

4.658271312713623
3.7861011028289795
3.1342854499816895
2.658357620239258
2.6050984859466553
2.5792124271392822
2.5388400554656982
2.433148145675659
2.4081239700317383
2.496692657470703


In [ ]:
print(decode(model.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=200)[0].tolist()))


WoXm?LGD.i:'MmlEoP;RaIvcrAPM$pMXNjPvObWUQpQ&zcN3:Os?yNSaeoG3XFQf3mPe-bbn;wjGAzLglre?BPu-XhDSi'';RgwQRpqeWuBHHwhiy;barF-WJK3DGq3-aReC.3JKFKmWECWDzQOTK?N
UOrl;jYttWpeC3&$UcJDS,j.'M?lVNLX;vqFzThicHFikvdn


## Finally Let's start talking about `self attention`

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


###   Hyperparameters
batch_size = 16
block_size = 128
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu' #device agnostic code
eval_iters = 200
n_embd = 240
n_head = 6
n_layer = 6
dropout = 0.2


In [ ]:
diff_model_loss = {}

In [ ]:
def get_batch(split):
  data = train_data if split=="train" else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x.to(device), y.to(device)


In [ ]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias = False)
    self.query = nn.Linear(n_embd, head_size, bias = False)
    self.value = nn.Linear(n_embd, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones((block_size, block_size))))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)       # (B,T,head_size)
    v = self.value(x)     # (B,T,head_size)
    q = self.query(x)     # (B,T,head_size)
    wei = q @ k.transpose(-2,-1) * C**-0.5  ## (B,T,h) @ (B,T,h).T = (B,T,h) @ (B,h,T) = (B,T,T)
    ## masking
    wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)
    out = wei @ v
    return out

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)
  def forward(self,x):
    x = torch.cat([h(x) for h in self.heads], dim = -1)
    out = self.dropout(self.proj(x))
    return out

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(in_features = n_embd, out_features = 4*n_embd),
        nn.ReLU(),
        nn.Linear(in_features = 4*n_embd, out_features = n_embd),
        nn.Dropout(dropout)
    )
  def forward(self,x):
    return self.net(x)

In [ ]:
class Block(nn.Module):
  '''
  A Block intersperses commuincation and computation
  '''
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd//n_head
    self.sa = MultiHeadAttention(n_head, head_size) ## Communication Channels
    self.ffwd = FeedForward(n_embd) ## Computation
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)
  def forward(self,x):
    x= x + self.sa(self.ln1(x))
    x=x + self.ffwd(self.ln2(x))
    return x

In [ ]:

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size, n_embd):
    super().__init__()
    self.token_embedding_table = nn.Embedding(num_embeddings = vocab_size, embedding_dim=n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    # self.sa_heads = MultiHeadAttention(4, n_embd//4)  ## 4 head of 8-dimensional self-attention
    # self.ffwd = FeedForward(n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head =n_head) for _ in range(n_layer)])
    self.lm_head = nn.Linear(in_features = n_embd, out_features = vocab_size)
    self.ln_f = nn.LayerNorm(n_embd)

  def forward(self,idx, targets=None):
    # idx Shape: (B,T)
    B,T = idx.shape
    #idx and targets are both (B,T) tensor of integers
    tok_emb = self.token_embedding_table(idx)  #(B,T,C)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # (T,C)
    x = tok_emb + pos_emb  ## (B,T,C) + (T,C) -> Broadcasting => (B,T,C) + (B,T,C) = (B,T,C)
    # x= self.sa_heads(x)
    # x = self.ffwd(x)
    x = self.blocks(x) # (B,T,C)
    x = self.ln_f(x)   # (B,T,C)
    logits = self.lm_head(x) #(B,T,vocab_size)
    if targets is None:
      loss = None
    else :
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits , loss

  def generate(self, idx, max_new_tokens):
     #idx is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      #crop the idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      #get the predictions
      logits, loss = self(idx_cond)  ##similar to self.forward(idx)  as forward implements __call__ internally
      ## logits : shape -> (B,T,C)
      logits = logits[:, -1,:] #becomes (B,C)
      #apply softmax to get the probabilities
      probs = F.softmax(logits, dim=-1) ## (B,C)
      idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
      idx = torch.cat((idx, idx_next), dim=1) #(B,T+1)
    return idx



model = BigramLanguageModel(vocab_size = vocab_size, n_embd = n_embd).to(device)
xb, yb = get_batch('train')
logits, loss = model(xb, yb)
print(logits.shape)
print("loss = ",loss.item())

# print(decode(model.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([2048, 65])
loss =  4.314550876617432


In [ ]:
## before that let's create some more useful and optimized code

## for evaluating loss
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

  ## trainig loop
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
  ##every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
  #sample a batch of data
  xb,yb = get_batch('train')
  #evaluate the loss
  logits,loss =  model(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  # ###
  # if iter%2000==0:
  #   inp = input('Continue or Not: y/n')
  #   if inp=='y':
  #     break

context =  torch.zeros((1,1), dtype=torch.long, device = device)
# print(decode(model.generate(idx = context, max_new_tokens=200)[0].tolist()))



Step 0: train loss 1.5150, val loss 1.5150
Step 500: train loss 1.4826, val loss 1.4826
Step 1000: train loss 1.5133, val loss 1.5133
Step 1500: train loss 1.4256, val loss 1.4256
Step 2000: train loss 1.4908, val loss 1.4908
Step 2500: train loss 1.4153, val loss 1.4153
Step 3000: train loss 1.4841, val loss 1.4841
Step 3500: train loss 1.4108, val loss 1.4108
Step 4000: train loss 1.3526, val loss 1.3526
Step 4500: train loss 1.4350, val loss 1.4350


In [ ]:
print(decode(model.generate(idx = context, max_new_tokens=400)[0].tolist()))



Who, that thou'lvest I know under'd after, he
bid our kinsman'st bolded's bane respecteth,
So I'll not a thee confirence-mean with measure,
Insel and amendom pace, here is could
At cannot pleace one an inclarate!
That tears, will turn to the crown?

HORTENSIO:
Why matter entempt
For the strangers is no.

CORIOLANUS:
Then? be striked, good pow?

AUFIDIUS:
Chastionming rought,
When, my soul liege's 


In [ ]:
new_model_feature_name = 'Scaling it even more....'
diff_model_loss[new_model_feature_name] = estimate_loss()

In [ ]:
diff_model_loss

{'Scaling it even more....': {'train': tensor(1.3561), 'val': tensor(1.3561)}}

```py
{
  'Uniform Affinity': {'train': tensor(2.3619), 'val': tensor(2.3619)},
  'Single Head Attention':  {'train': tensor(2.2240), 'val': tensor(2.2240)},
  'Multi Head Attention': {'train': tensor(2.2880), 'val': tensor(2.2880)},
  'Feed Forward': {'train': tensor(2.1118), 'val': tensor(2.1118)},
  'Blocks of Attention and FeedForward': {'train': tensor(2.2323),'val': tensor(2.2323)},
  'Optimization': {'train': tensor(1.8434), 'val': tensor(1.8434)},
  'Scaling it even more....': {'train': tensor(1.5150), 'val': tensor(1.5150)},
  'Scaling it even more 2 ....': {'train': tensor(1.3561), 'val': tensor(1.3561)}


}
```


In [ ]:
torch.save(model.state_dict(), 'scratchGPT2.pth')